In [1]:
import numpy as np 
import time 
import matplotlib.pyplot as plt

In [2]:
# Hamiltonian operators for spin 1 Heisenberg chain

# Pauli Z for spin 1
Sz = np.zeros((3,3))
Sz[0,0] = 1
Sz[2,2] = -1

#Raising operator S^+
Splus = np.zeros((3,3))
Splus[0,1] = np.sqrt(2)
Splus[1,2] = np.sqrt(2)

#Lowering operator S^-
Sminus = np.zeros((3,3))
Sminus[1,0] = np.sqrt(2)
Sminus[2,1] = np.sqrt(2)

#Identity operator
I = np.eye(3)

In [3]:
#Density Matrix Renormalization-Group (DMRG) algorithm parameters
N_iterations = 40
bond_dim = 10
degeneracy_tolerance = 1e-10

for iteration in range(1,N_iterations+1):
    if iteration == 1:
        #System identity
        I_system = I

        #System Hamiltonian
        H_system = I_system*0

        #Boundary operators with system
        Szboundary_system = Sz
        Splusboundary_system = Splus
        Sminusboundary_system = Sminus

        #Environment identity
        I_environment = I

        #Environment Hamiltonian
        H_environment = I_environment*0

        #Boundary operators with environment
        Szboundary_environment = Sz
        Splusboundary_environment = Splus
        Sminusboundary_environment = Sminus

        #if you are reading this for the first time, don't continue with the else
        #statement immediately but continue with what's after the first iteration first.

    else:

        dimension_left = np.shape(I_left)[0]
        dimension_right = np.shape(I_right)[0]

        #reshape the statevector into a left and right components as a matrix
        psi_matrix = np.reshape(psi, (dimension_left, dimension_right))

        #starting with the left side
        ReducedDensityMatrix_left = np.matmul(psi_matrix,psi_matrix.conj().T)
        evals_left, evecs_left = np.linalg.eigh(ReducedDensityMatrix_left)

        #change of basis matrix
        Orthogonal_left = np.array(evecs_left[:,-1])
        for k in range(1,min(bond_dim,len(evals_left))):
            Orthogonal_left = np.vstack([Orthogonal_left, evecs_left[:,-1-k]])

        #check for degenerate states
        rows_Orthogonal_left = np.shape(Orthogonal_left)[0]
        for k in range(rows_Orthogonal_left,len(evals_left)):
            if abs(evals_left[-rows_Orthogonal_left] - evals_left[-1-k]) < degeneracy_tolerance:
                Orthogonal_left = np.vstack([Orthogonal_left, evecs_left[:,-1-k]])
            else:
                break

        #repeat for the right side
        ReducedDensityMatrix_right = np.matmul(psi_matrix.T,psi_matrix.conj())
        evals_right,evecs_right = np.linalg.eigh(ReducedDensityMatrix_right)

        #change of basis matrix
        Orthogonal_right = np.array(evecs_right[:,-1])
        for k in range(1,min(bond_dim,len(evals_right))):
            Orthogonal_right = np.vstack([Orthogonal_right, evecs_right[:,-1-k]])

        #check for degenerate states
        rows_Orthogonal_right = np.shape(Orthogonal_right)[0]
        for k in range(rows_Orthogonal_right,len(evals_right)):
            if abs(evals_right[-rows_Orthogonal_right] - evals_right[-1-k]) < degeneracy_tolerance:
                Orthogonal_right = np.vstack([Orthogonal_right, evecs_right[:,-1-k]])
            else:
                break

        #change of basis operations using the new basis matrices on the left and right for the system and environment
        I_system = np.matmul(np.matmul(Orthogonal_left, I_left),Orthogonal_left.conj().T)

        H_system = np.matmul(np.matmul(Orthogonal_left,Hamiltonian_left),Orthogonal_left.conj().T)

        
        Szboundary_system = np.matmul(np.matmul(Orthogonal_left,Szboundary_left),Orthogonal_left.conj().T)
        Splusboundary_system = np.matmul(np.matmul(Orthogonal_left,Splusboundary_left),Orthogonal_left.conj().T)
        Sminusboundary_system = np.matmul(np.matmul(Orthogonal_left,Sminusboundary_left),Orthogonal_left.conj().T)
        
        I_environment = np.matmul(np.matmul(Orthogonal_right,I_right),Orthogonal_right.conj().T)
        
        H_environment = np.matmul(np.matmul(Orthogonal_right,Hamiltonian_right),Orthogonal_right.conj().T)
        
        Szboundary_environment = np.matmul(np.matmul(Orthogonal_right,Szboundary_right),Orthogonal_right.conj().T)
        Splusboundary_environment = np.matmul(np.matmul(Orthogonal_right,Splusboundary_right),Orthogonal_right.conj().T)
        Sminusboundary_environment = np.matmul(np.matmul(Orthogonal_right,Sminusboundary_right),Orthogonal_right.conj().T)

    #update operators on the left
    I_left = np.kron(I_system,I)
    Hamiltonian_left = np.kron(H_system, I)

    #add new terms in the left Hamiltonian after appending the new spin to the left side
    Hamiltonian_left += np.kron(Szboundary_system,Sz) + 0.5*( np.kron(Splusboundary_system,Sminus) + np.kron(Sminusboundary_system,Splus))

    #new left boudary operators after adding the new spin
    Szboundary_left = np.kron(I_system, Sz)
    Splusboundary_left = np.kron(I_system, Splus)
    Sminusboundary_left = np.kron(I_system, Sminus)

    #update operators on the right
    I_right = np.kron(I,I_environment)

    Hamiltonian_right = np.kron(I,H_environment)
    Hamiltonian_right += np.kron(Sz, Szboundary_environment) + 0.5*(np.kron(Splus, Sminusboundary_environment) + np.kron(Sminus,Splusboundary_environment))

    Szboundary_right = np.kron(Sz, I_environment)
    Splusboundary_right = np.kron(Splus, I_environment)
    Sminusboundary_right = np.kron(Sminus, I_environment)

    #full system Hamiltonian
    Hamiltonian_full = np.kron(Hamiltonian_left, I_right)
    Hamiltonian_full += np.kron(I_left, Hamiltonian_right)
    Hamiltonian_full += np.kron(Szboundary_left,Szboundary_right) + 0.5*(np.kron(Splusboundary_left, Sminusboundary_right) + np.kron(Sminusboundary_left,Splusboundary_right))

    #psi for the density matrix calculation
    Energies, States = np.linalg.eigh(Hamiltonian_full)
    psi = States[:,0]

    #Sanity check prints
    GS_energy = Energies[0]
    N_spins = 2+2*iteration
    print('#spins = ', N_spins)
    print('Ground state energy =', GS_energy)
    print('GS energy per spin =', GS_energy/N_spins)
    print('dim of SB = ', np.shape(Hamiltonian_full)[0])
    print('\n')


#spins =  4
Ground state energy = -4.64575131106459
GS energy per spin = -1.1614378277661475
dim of SB =  81


#spins =  6
Ground state energy = -7.370274969424608
GS energy per spin = -1.228379161570768
dim of SB =  729


#spins =  8
Ground state energy = -10.124360116756387
GS energy per spin = -1.2655450145945484
dim of SB =  1296


#spins =  10
Ground state energy = -12.893433750076662
GS energy per spin = -1.2893433750076662
dim of SB =  1296


#spins =  12
Ground state energy = -15.671397528300991
GS energy per spin = -1.3059497940250826
dim of SB =  1296


#spins =  14
Ground state energy = -18.454955999828886
GS energy per spin = -1.3182111428449204
dim of SB =  1296


#spins =  16
Ground state energy = -21.242558912947757
GS energy per spin = -1.3276599320592348
dim of SB =  1296


#spins =  18
Ground state energy = -24.032836487997738
GS energy per spin = -1.335157582666541
dim of SB =  1296


#spins =  20
Ground state energy = -26.825295754471842
GS energy per spin = -1.3412